# RSA

Rivest Shamir Adleman (RSA) jest jednym z pierwszych algorytmów asymetrycznych. Został zaprojektowany w 1977 roku i jest używany do dzisiejszego dnia. Zyskał akceptację zarówno NISTu jak i organizacji ISO/IEC oraz RFC. RSA posiada parę kluczy - publiczny oraz prywatny. Publiczny klucz może być znany każdemu i służy on do operacji szyfrowania. Klucz prywatny jest znany tylko i wyłącznie instancji, która klucze generowała. Ta sama instancja jako jedna jedyna ma możliwość odszyfrowania kryptogramów.

RSA umożliwia także tworzenie podpisów cyfrowych (z ang *Digital Signatures*, czyli *DS*). Podpis cyfrowy to dodatkowy blok informacji dołączony do wiadomości, który zapewnia:
1. *Integrity* - integralność wiadomości, czyli potwierdzenie, że nie była ona w żaden sposób modyfikowana.
2. *Authentication* - autentykacje podpisującego, czyli potwierdzenie jego tożsamości.
3. *Non-repudiation* - czyli wysyłający podpisaną wiadomość nie ma możliwości zaprzeczenia faktu, że to on ją podpisał, natomiast otrzymujący wiadomość nie ma możliwości zaprzeczenia faktu, iż to on ją zweryfikował. 

Samo haszowanie wiadomości zapewnia tylko *integirty*, natomiast utworzenie kodu MAC (*Message Authentiaction Code*) zapewnia jedynie *integrity* oraz *authentiaction*.

Tworzenie podpisu cyfrowego z wykorzystaniem RSA wygląda odwrotnie niż komunikacja szyfrowana. To znaczy: podpis tworzony jest z wykorzystaniem klucza prywatnego - a więc tylko instancja generująca klucze może wiadomość podpisać. Weryfikacja odbywa się z wykorzystaniem klucza publicznego - czyli każda instancja, której nasz klucz udostępnimy, może podpis zweryfikować. 

Na początek zaimportujmy niezbędne biblioteki.

In [1]:
!pip install pycryptodome

In [2]:
import Crypto.Util.number as cu
import hashlib as hl
import math

**Zadanie 1**

Odpowiedz na poniższe pytania. Rozważ sytuację, w której dwie instancje komunikują się ze sobą, a trzecia próbuje dokonywać nieautoryzowanych zmian w treści wiadomości na drodze jej przesyłu.
1. Dlaczego haszowanie zapewnia tylko integrity? Podpowiedź: czy haszowanie uwzględnia jakikolwiek klucz prywatny?
2. Dlaczego kod MAC nie zapewnia *non-repudiation*? Co (niepożądanego) może dokonać odbierający wiadomość i atakujący komunikację? Podpowiedź: czy kody MAC, w szczególności popularny kod HMAC - który zakłada użycie klucza prywatnego - w jakikolwiek sposób weryfikuje KTO i DO CZEGO go użył? Kto dysponuje, a kto NIE dysponuje kluczem prywatnym HMAC?
3. Dlaczego podpis cyfrowy zapewnia wszystkie te trzy cechy?

## Generowanie kluczy

Algorytm generowania kluczy RSA może zostać przedstawiony w następujący sposób:

1) Znajdź dwie różne i kryptograficznie bezpieczne liczby pierwsze.

2) Oblicz $n = p * q$.

3) Oblicz $f = (p - 1) * (q - 1)$.

4) Znajdź dowolne $e$, takie, że $1 < e < f$ oraz $GCD(f, e) = 1$. GCD to największy wspólny dzielnik. Para $(e, n)$ to jest **klucz publiczny**.

5) Oblicz $d = e^{-1}$ mod $f$. Para $(d, n)$ to **klucz prywatny**, przy czym tajne jest tylko $d$. 

W ten sposób generowane parametry byłyby matematycznie poprawne, lecz kryptograficznie niebezpieczne. Ustalmy więc, że chcemy aby nasz klucz publiczny był odpwowiednio długi. Będzie to długość bitowa parametru $n$, oznaczmy ją jako $nlen = 2048$. Parametr $nlen$ zawsze przyjmuje parzyste wartości. Mając to założenie, musimy (**uwzględniając wszystkie założenia z algorytmu generowania kluczy**) dodatkowo zapewnić, że:

1. $65537 ≤ e < 2^{256}$
2. $LCM(p - 1, q - 1) \geq e * 2^{nlen/2}$
3. $2^{(nlen - 1)//2} < p < 2^{nlen/2}$
4. $2^{(nlen - 1)//2} < q < 2^{nlen/2}$
5. $|p - q| > 2^{(nlen/2) - 100}$

Gdzie LCM oznacza *Least Common Multiple*, czyli najmniejszą wspólną wielokrotność. Funkcję LCM znajdziesz w bibliotece math. Do potęgowania **nie używaj** pythonowej notacji "**", tylko metody pow() - przetestuj obie te metody obliczania potęgi i porównaj wydajność (zadanie opcjonalne). Do obliczania wartości bezwzględnej użyj metody abs() - również standardowa metoda pythona. Resztę niezbędnych metod znajdziesz w bibliotece [Crypto.Util.number](https://pycryptodome.readthedocs.io/en/latest/src/util/util.html) zaimpoertowanej jako cu. Opis powyższych założeń możesz znaleźć w [tym](https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-56Br2.pdf) dokumencie NIST-owym.

**Zadanie 2**

Uwzględniając wszystko powyższe, napisz metodę/metody generujące wartości $n$, $e$ oraz $d$.

In [177]:
nlen = 2048

e = 65537
p = cu.getStrongPrime(int(nlen/2), e)
q = cu.getStrongPrime(int(nlen/2), e)
n = p * q
f = (p - 1) * (q - 1)
d = cu.inverse(e, f)

In [178]:
conditions = [
    cu.GCD(e, f) == 1,
    2**((nlen-1) // 2) < p,
    2**((nlen-1) // 2) < q,
    abs(p-q) > 2**(nlen/2 - 100)
]

len([condition for condition in conditions if condition]) == len(conditions)

True

## Naiwne szyfrowanie i deszyfrowanie


Naiwny algorytm szyfrowania wiadomości **M** z wykorzystaniem RSA:

1) Zakoduj $M$ jako liczbę.

2) Oblicz: $C = M^e$ mod $n$.

Naiwny algorytm deszyfrowania kryptogramu **C** z wykorzystaniem RSA:

1) $M = C^d$ mod $n$.

2) Zdekoduj wiadomość $M$ do jej pierwotnej postaci (np. stringa). 


**Zadanie 3**

Napisz metody szyfrujące i deszyfrujące wiadomość $M$ zgodnie z powyższym algorytmem. Zaszyfruj wiadomość, zdeszyfruj i wypisz oryginalny tekst na ekranie. Odpowiedz na pytanie: jaki warunek musi spełniać liczbowa reprezentacja wiadomości $M$, aby można ją było poprawnie zaszyfrować i zdeszyfrować?

In [179]:
import Crypto.Math.Numbers as cn

In [180]:
def rsa_encode(m):
    eM = cn.Integer.from_bytes(m)
    return eM.inplace_pow(e, n).to_bytes()

def rsa_decode(m):
    C = cn.Integer.from_bytes(m)
    dM = C.inplace_pow(d, n)
    return dM.to_bytes()

In [181]:
M = "Ala ma kota".encode('utf-8')

eM = rsa_encode(M)
print(eM)
print(rsa_decode(eM))

b'Y\xd6\xb1\xc0\x9e\xa5%\xf6\x82 yz\xf8\xa1\x97\xef-\xba\xe4d\xc2\'\xdfD\xf1t\xb8\x02\xc9-"\xcb;Iz\x8a\xb4Q\xc3u\xf6\xfe\x07x\x1fr\x0c\xb0\xac\xc2#2\xa55p\xb6\n\xb1\xea\xb1\xe6Q9h\xc6{\xdf\x06\xff\x1a7V\xe1\'\x8bg\xb9]\xbf+\x80C\xf5#h\x13:Kh\t\x9a.*\x8c\xd2\xcbBiv[\xe9\xc0M\x80\xdb,Z\xb7>\xe8\xbc\xbc\xef\xc9\xc7\x01\xfeR~y\xf9\xa0_\xf1\xabn7\xef\x13\xb3\xf9\x8d\x9b_\xf3\x8a\xb7k\xfe\xac\xa5|r\xe3\xf1\xba\xa9\x02FM\xb3\xb1\r\xd7-u\xa37\x07\xa5\x1ejBd\x8c\x82\xc7\x1b\x94u\xbe\x95W\xd8\x8e\x8cQ\xe3\xb7T7\xe2\xe1\x16\x07\x94r~\x14u\x0b\x95\xa4&\xd5`P\xcc e\xf0e\xf7;\xe7\x1eQ\x87\xa5vW\x9fF\x03\x8bm3\xe2*\xb8\x83\xc4\x04\xc4:cR\\\xcf\xc1\x10\xb5;\xcd\xd5\xcd\xa32\xdf\xadL\x1f?3W+\xc7l\xda\x85gg\xe1}\xaav'
b'Ala ma kota'


## Naiwny schemat podpisu cyfrowego

Naiwna metoda tworzenia podpisu z wiadmości $M$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $SIG = h^d$ mod $n$.

4) Wyślij parę $(M, SIG)$ weryfikującemu.

Naiwna metoda weryfikacji podpisu $(M, SIG)$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $VER = SIG^e$ mod $n$.

4) Jeżeli $VER = h$, weryfikacja przebiegła pomyślnie, a w przeciwnym razie niepomyślnie.

**Zadanie 4**

Zaimplementuj naiwną metodę tworzenia i weryfikowania podpisu cyfrowego RSA.

In [182]:
def rsa_sign(m):
    h = hl.sha256(m).digest()
    nh = cn.Integer.from_bytes(h)
    SIG = nh.inplace_pow(d, n)
    return SIG.to_bytes()

def rsa_verify(m, SIG):
    VER = cn.Integer.from_bytes(SIG).inplace_pow(e, n)
    return VER == cn.Integer.from_bytes(hl.sha256(m).digest())

In [183]:
SIG = rsa_sign(M)
print(SIG)
print(rsa_verify(M, SIG))

b'\\u\xf5\xc4>\xd3\xb0\xe3a\x8d@\x87\x9d\x8f\xd2t\x84\xc2v\xc3\xef\\ \xdc\x06\x9c>\xb2\x90\x80\xa9\x83\xb5\x82\xa0I\x0f\x95\xa8\xe9\xdf\xb5\xd04\xa0O\x1d\xdcY\xf0\x0b\'\xc3\xf2\xcc\xbas9\x0cH5b\x86\xf5\x03\xc3\xa8ls\x0f\x17Y\xea\x08\xab^\x98\x97\x92?\xa0P8\xa6PS}\xdfE\x88k\xc8\x0c\x8b\xba\x1b\xac\x94\xef\x7fX\x00\x12\x87\x8e8\xa6\xc6d\r\x95\x8d1\xa9a\xb1\xe9W\xb09\xe1\xef\x85^\xb1\x94\xc1_H\xb3E\xfe\xc1\xdd$\xdb\xfa\x0f\xd2\xe7\x95\x9b5\xf1z\x9d\xb2\xb5\xcb\x0c@&^\xd0\xeb\x82\xb0 \t\xe1\xf2YMb-\xdb\x15h\xb33\xd5\xaa=\n\x03\x9c\xb9O(\x08\x83\x0e\x00\xea\xb1\xe4\xee\xc4.\xa62\xab+\xbd\xde\xc0\x97w\xfb\xdc_\xbcYc"\xa0}B\x80\x81\x07\xd3\x0b\xb6I\x96e\x0e*\x0e\xcbT)\xfdqr\xaf\xebG\xf7\xe0\x14\\r\xc0\xc6\x0b6X\xfe"O\xb2X\xfc\xce?\r\x98!\x9f\x87~\xfc\xc8b'
True


## MGF 1

W dalszej części laboratoriów będziemy potrzebowali generować maskę. Jedynym zatwierdzonym algorytmem który do tego służy jest *Mask Generation Function 1*, opisany w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017). Jest on stosunkowo prosty. 

Parametry wejściowe:

1) M - bajty wiadomości.

2) len - pożądana długość zwórconej maski w bajtach.

3) H - wybrana funkcja skrótu, zwracająca $n$ bitowy skrót. Niech to będzie SHA-256. Dla wygody przyjmijmy też, że $hlen = n / 8$ oznacza liczbę bajtów zwracaną przez naszą funkcję skrótu.

Wyjściem funkcji są bajty tworzące maskę.

Algorytm MGF-1:

1) Dla 32-biotwego integera $i = 0, ..., ⌈ \frac{len}{hlen}⌉ - 1$ wykonuj kroki 2 i 3.

2) Oblicz tmp = H(M || i). Znak || to konkatenacja i chodzi tu o bajty wiadomości M oraz reprezentację w bajtach 32-bitowego itegera $i$.

3) Oblicz output = output || tmp.

4) Zwróc $len$ wiodących bajtów zmiennej output.

**Zadanie 5**

Zaprogramuj i przetestuj dla dowolnych wartości funkcję MGF1.

In [184]:
def octet_string(i, octets=4):
    s = ""
    s += str((i >> 24) & 0xff).zfill(3)
    s += str((i >> 16) & 0xff).zfill(3)
    s += str((i >> 8) & 0xff).zfill(3)
    s += str(i & 0xff).zfill(3)
    return s.encode("utf-8")

def mgf1(M, l):
    out = b''
    for i in range(0, cu.ceil_div(l, 32)):
        tmp = hl.sha256(M + octet_string(i)).digest()
        out += tmp
    return out[:l]

## OAEP

Nasz schemat ma na ten moment jedną sporą wadę, mianowicie rozmiar szyfrowanej wiadomości może być zbyt mały, czyniąc algorytm mniej bezpiecznym. Aby tego uniknąć, używamy algorytmu paddingu opisanego w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017#section-8), który zwie się *Optimal Assymetric Encryption Padding*.

### OAEP encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - długość liczby $n$ wyrażona w bajtach.

3) $mlen$ - długość wiadomości wyrażona w bajtach.

4) $M$ - bajty wiadomości.

5) $mgf1$ - Mask Generation Function 1.

Algorytm:

1) Jeżeli $mlen > k - 2*hlen - 2$ zwróc błąd.

2) Oblicz: $lHash = H("")$.

3) Wygeneruj tablicę bajtów $PS$ składającą się z $k - mlen - 2*hlen - 2$ bajtów o wartości 0x00. Rozmiar $PS$ może wynosić 0.

4) Oblicz: $DB = lHash || PS || 0x01 || M$. Długość $DB$ powinna wynosić $k - hlen - 1$ bajtów.

5) Wygeneruj losową tablicę bajtów $seed$ o rozmiarze $hlen$.

6) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

7) Oblicz: $maskedDB = DB ⊕ dbMask$.

8) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

9) Oblicz: $maskedSeed = seed ⊕ seedMask$.

10) Oblicz: $EM = 0x00 || maskedSeed || maskedDB$. Długość $EM$ powinna wynosić $k$.

11) Zwróc $EM$.

### OAEP decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - rozmiar EM wyrażony w bajtach.

3) $mgf1$ - Mask Generation Function 1.

4) $EM$ - bajty zakodowanej wiadomości.

Algorytm:

1) Rozpakuj tablicę bajtów $EM$. Jej pierwszy bajt (najbardziej znaczący) przypisz do $Y$. Kolejne $hlen$ bajtów przypisz do $maskedSeed$, resztę do $maskedDB$. Czyli $EM = Y || maskedSeed || maskedDB$.

2) Oblicz: $lHash = H("")$.

3) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

4) Oblicz: $seed = maskedSeed ⊕ seedMask$.

5) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

6) Oblicz: $DB = maskedDB ⊕ dbMask$.

7) Rozpkauj tablicę bakjtów $DB$. Pierwsze (najbardziej znaczące) $hlen$ bajtów przypisz do zmiennej $lHash'$. Następne $k - mlen - 2*hlen - 2$ bajtów do PS. Kolejny pojedynczy bajt powinien wynosić 0x01, jeżeli jest inaczej zwróć błąd i **zakończ działanie**. Resztę bajtów przypsiz do zmiennej $M$. Czyli: $DB = lHash' || PS || 0x01 || M$. 

8) Jeżeli $Y \neq 0x00$ zwróć błąd i **zakończ działanie**.

9) Jeżeli $lHash \neq lHash'$ zwróć błąd i **zakończ działanie**.

10) Zwróc $M$.

**Zadanie 6**

Zaproogramuj kodowanie i dekodowanie OAEP. Zmodyfikuj algorytm szyfrowania RSA, tak, aby przed zaszyfrowaniem wiadomość była paddingowana. Zmodyfikuj algorytm deszyfrowania tak, aby po zdeszyfrowaniu konieczne było wywołanie metody dekodowania OAEP w celu odzyskania wiadomości.

In [185]:
def xor(vs):
    return vs[0]^vs[1]

class Bytes():
    def __init__(self, _bytes):
        self._bytes = _bytes
        
    def __xor__(self, other):
        return Bytes(bytes(map(xor, zip(self._bytes, other._bytes))))

In [186]:
import random

def randombytes(l):
    return bytes([random.randrange(0, 256) for _ in range(l)])

In [187]:
def H(v):
    return hl.sha256(v).digest()

In [262]:
def oaep_padding(m):
    hlen = 32
    k = nlen // 8
    mlen = len(m)

    if mlen > k - 2*hlen - 2:
        raise Exception("Błąd")

    lHash = H(b"")
    PS = b"\x00" * (k - mlen - 2*hlen - 2)
    DB = lHash + PS + b"\x01" + m

    if len(DB) != k-hlen-1:
        raise Exception("Błąd")

    seed = randombytes(hlen)
    dbMask = mgf1(seed, k - hlen - 1)
    maskedDB = (Bytes(DB) ^ Bytes(dbMask))._bytes
    seedMask = mgf1(maskedDB, hlen)
    maskedSeed = (Bytes(seed) ^ Bytes(seedMask))._bytes
    EM = b"\x00" + maskedSeed + maskedDB

    if len(EM) != k:
        raise Exception("Błąd")
        
    return EM

def oaep_remove(EM, mlen):
    hlen = 32
    k = nlen // 8

    #może uciąć początkowe \x00
    EM = b"\x00"*(k - len(EM)) + EM

    Y, maskedSeed, maskedDB = EM[0], EM[1:hlen+1], EM[hlen+1:]
    lHash = H(b"")
    seedMask = mgf1(maskedDB, hlen)
    seed = (Bytes(maskedSeed) ^ Bytes(seedMask))._bytes
    dbMask = mgf1(seed, k - hlen - 1)
    DB = (Bytes(maskedDB) ^ Bytes(dbMask))._bytes

    if DB[hlen+(k-mlen-2*hlen-2)] != 1:
        raise Exception("Błąd")
    lHash_p, PS, dM = DB[:hlen], DB[hlen:hlen+(k-mlen-2*hlen-2)], DB[hlen+(k-mlen-2*hlen-2)+1:]

    if Y != 0:
        raise Exception("Błąd")

    if lHash != lHash_p:
        raise Exception("Błąd")

    return dM

In [263]:
M = "Ala ma kota".encode('utf-8')

pM = oaep_padding(M)
print(pM)
eM = rsa_encode(pM)
print(eM)
dM = rsa_decode(eM)
print(dM)
m = oaep_remove(dM, len(M))
print(m)

b'\x00h\x82\xb3\x16k\xb3\xd0\xbe\xa4\xaf\xe4l\xeb\x1f\xc6\x00%J\xe7\x0cM\nU\x88~|T\x80;\xd9\x97>\xc6\xb1"\x81}\xb5\xc49G\xbdt\xc3\xeb\xec\xa5R!+\xe6\xa2\xc0\xfc\xbes`\xe4oZ\xdb|\xa5\xd8X\xd9\xef\xc1\x93r\xa2\x11Q4S\xba\xdc\xbcj\xd1\xba\x95\xeeg\xd2\x1eg\x9e\x90K"4\x80\x16\xfbv%\x97\x8c\xac\xcbO\x85\x8c.\xaa2\xf3\xc7\xd9\x96#\xbe\xb0\xba\xda\xf0!U\xe4\x94\xd2\xebCa\x15K\xc5@\xe9\x95\x01]"\xf1,\xd9\x861\xbc9\xba\xa2:\x82+\xa4A\xcc:n5\x1b\xed\x92\x7f\xd1\x1dB\xb7ZJ\xa7\xc9\xa3\x1f7{\xa1-\xdc\xfb\xc5\xd0\x85b\xe9\x16\xdbm=\x86wo\x1b4\xf9u\xbbV\x7fw\xb4]S=\x8b3\x8d\xd5E\xdfN\xdaL\x98$\x05\x8fCC\x074Sx~\xd0\x87\xfeb\x11\t\xe6Q\xe2\xab\x9b\xd6\x86\t\'\xad\r\xcd\x14o\xa5"t\xfd\x98\xa3W\xa1\x14\xd9\nU\x1c}\x19-\x0bs\xac'
b'\xa1eL\xe4m-\x7f\x8d[\xd2\x0co\xe0j\xafvJ4\xc7D\xbe\x13\xc4\xfb\x04\x89\xb2^@\xefe\x888\xa1\xe4CZ\x04u%a\x10\x9e\xacy\xe3\x99YHL\x14\x81*\xdfN\x8d\xa2\x85\xdaE\xbf\xc66\xe4dR\xabKh\xebiL\xad)\x7f\xb2\x00\x12\xfc\x15f\x12\xe5u1@\x1d\xfd\xd6\xee\xe0\xd1\x95i\xef\x993\xde\x9c\x8

## EMSA - PSS

Utworzenie bezpiecznej sygnatury RSA wymaga zastowania algorytmu *Encoding Method for Signature with Appendix - Probabilistic Signature Scheme* .

### EMSA encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $M$ - bajty wiadomości do podpisania.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - pożądana długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.


Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd**.

3) Wygeneruj tablicę losowych bajtów $salt$ o długości $slen$.

4) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

5) Oblicz: $mHash' = H(M')$.

6) Wygeneruj tablicę $PS$ składającą się z bajtów 0x00 o długości $emlen - slen - hlen - 2$.

7) Oblicz: $DB = PS || 0x01 || salt$. Długość $DB$ powinna wynosić $emlen - hlen - 1$ bajtów.

8) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

9) Oblicz: $maskedDB = DB ⊕ dbMask$.

10) Ustaw $8 * emlen - emBits$ **najbardziej znaczących** bitów $maskedDB$ na wartości 0.

11) Oblicz: $EM = maskedDB || mHash' || 0xbc$.

12) Zwróć $EM$.


In [257]:
def emsa_sign(m):
    hlen = 32
    slen = hlen
    emBits = nlen - 1
    emlen = nlen // 8

    mHash = H(m)

    if emlen < hlen + slen + 2:
        raise "Błąd"

    salt = randombytes(slen)
    M_p = b"\x00"*8 + mHash + salt
    mHash_p = H(M_p)
    PS = b"\x00"*(emlen - slen - hlen - 2)
    DB = PS + b"\x01" + salt
    dbMask = mgf1(mHash_p, emlen - hlen - 1)
    maskedDB = (Bytes(DB) ^ Bytes(dbMask))._bytes

    zeroBytes = (8 * emlen - emBits) // 8
    reminderZeroBits = (8 * emlen - emBits) - 8*zeroBytes
    maskedDB = bytes([0 if i < zeroBytes else b for i,b in enumerate(maskedDB)])
    reminderByte = maskedDB[zeroBytes] & (0xff >> reminderZeroBits)
    maskedDB = bytes([reminderByte if i == zeroBytes else b for i,b in enumerate(maskedDB)])

    return maskedDB + mHash_p + b"\xbc"

## EMSA decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $EM$ - sygnatura wiadomości $M$.

4) $M$ - bajty wiadomości do weryfikacji.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.

Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd weryfikacji**.

3) Jeżeli ostatni bajt (najmniej znaczący) $EM$ nie ma wartości 0xbc **zakończ i zwróć błąd weryfikacji**.

4) Podstaw $emlen - hlen - 1$ najbardziej znaczących bajtów do $maskedDB$ oraz kolejne $hlen$ bajtów do $mHash'$.

5) Jeżeli $8 * emlen - emBits$ najbardziej znaczących bitów $maskedDB$ nie ma wartości 0, **zakończ i zwróć błąd weryfikacji**.

6) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

7) Oblicz: $DB = maskedDB ⊕ dbMask$.

8) Ustaw $8 * emlen - emBits$ najbardziej znaczących bitów $DB$ na 0.

9) Jeżeli $emlen - hlen - slen - 2$ najbardziej znaczących bajtów $DB$ nie posiada wartości 0x00 lub gdy bajt na pozycji $emlen - hlen - slen - 1$ (licząc od najbardziej znaczącego) nie posiada wartości 0x01 **zakończ i zwróć błąd weryfikacji**.

10) Przypisz do zmiennej $salt$ dokładnie $slen$ najmniej znaczących bajtów $DB$.

11) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

12) Oblicz $mHash'' = H(M')$.

13) Jeżeli $mHash' \neq mHash''$ **zakończ i zwróć błąd weryfikacji**, w przeciwnym razie **weryfikacja powiodła się**.


In [258]:
def emsa_verify(m, EM):
    mHash = H(m)

    if emlen < hlen + slen + 2:
        return False

    if EM[-1] != 0xbc:
        return False

    maskedDB, mHash_p = EM[:emlen-hlen-1], EM[emlen-hlen-1:emlen-hlen-1+hlen]

    zeroBytes = (8 * emlen - emBits) // 8
    reminderZeroBits = (8 * emlen - emBits) - 8*zeroBytes
    if sum(maskedDB[:zeroBytes]) + (maskedDB[zeroBytes] & (0xff << (8 - reminderZeroBits))) != 0:
        return False

    dbMask = mgf1(mHash_p, emlen - hlen - 1)
    DB = (Bytes(maskedDB) ^ Bytes(dbMask))._bytes

    zeroBytes = (8 * emlen - emBits) // 8
    reminderZeroBits = (8 * emlen - emBits) - 8*zeroBytes
    DB = bytes([0 if i < zeroBytes else b for i,b in enumerate(DB)])
    reminderByte = DB[zeroBytes] & (0xff >> reminderZeroBits)
    DB = bytes([reminderByte if i == zeroBytes else b for i,b in enumerate(DB)])

    if sum(DB[:emlen - hlen - slen - 2]) != 0 and DB[emlen - hlen - slen - 1] != 1:
        return False

    salt = DB[-slen:]
    M_p = b"\x00"*8 + mHash + salt
    mHash_pp = H(M_p)

    if mHash_p != mHash_pp:
        return False
        
    return True

**Zadanie 7**

Zaprogramuj kodowanie i dekodowanie EMSA, a następnie popraw algorytmy tworzenia i weryfikacji podpisu cyfrowego RSA. Tworzenie podpisu powinno wyglądać tak, że wiadomość najpierw jest kodowana z wykorzystaniem EMSA, a później tworzony jest popdis z wykorzystaniem klucza prywatnego. Dekodowanie powinno wyglądać tak, że najpierw używany jest klucz publiczny do odtworzenia podpisu EMSA, a następnie wykorzystywane jest dekodowanie EMSA w celu weryfikacji.

In [259]:
m = "Ala ma podpis".encode('utf-8')

EM = emsa_sign(m)
print(EM)
SIG = rsa_encode(EM)
print(SIG)
VER = rsa_decode(SIG)
print(VER)
print(emsa_verify(m, EM))

b'_\x1f|b\t\x9cv\xb0\xc9\xa3v[D\xe0\xb0JL\x17\xb5\x06&\xf2\xee\xbc\xe7ZW\x17OY\xcd\xc4E3\xd1\x02DE\xe6k\x8f.\xdcB\xbc\x07\x0b\xeat\x92 T\x98\xb8a\xa1Hj\xcd\x13\x01\xa18\xd18\xb7\xa7\x80\x83\xe4\xf3\xdb\xc4C\xf4\x1fm\xab\xd3F\x1f\x1e\x15\xe8\xb7\xf1\xa0+\\l\x84a7K\x85\xc7M>\xe0O9\xe3\xd4>\xe0\xb4m\xf8\xfe\xdeO\xba\x9f\xa9d\xff\xb1-JW\xab\xacg\x03\x1a\xf8N\xd7\x89\xa9Pe\xd9\xba\x85Gq\xd0A6+\x06\x88J\x9a\x814\xa7\xe7\x12\x9a\xea\x88\xc1\xfa\xbc^\xa0\x16\\\xac\xb03\xaa\xfe \xe0\x96\xcc\x0c\xb7\xbf\xc0\xe4\x96\xebP\xa5\xd0_S\x16}\xd4\xfc6\x14\xf2R\x8b\xf5\xed\x9ca^\xeb\xe1@8\x1a\x83\x86\xb1\xd7f\xa9\r\x82\x81\x88\xedA\x8f\x97\xabdeL\x12{m3\x95\x8cnY\xf0^-\xf9\xd7\xf5\x92\xecw\xdesd\xf0i\xd8\xb6\xfdmA\x91\x8fL3\xc1\x97\xab|S\xe7\xbc'
b'\'r\x05&\x95Y\xe5\xe2\xb6\x03\xc1\x86,k\x0b\x1d\xe4\xac\xa4/\xb7)\x91M\xfc\xb6\x88\xa1\x9a\xadM\x8b\x99\xa4*p\xdb\xa5:\x1ez3y\xccI>\xf7\xf8\x96\xba\x14\xfe\x15\xd6N\xf2\xb1\xe5\xc0\x13\xd3\xef\x82\xd3]\xcb\xe1\x88\xdb7]\xb3z\xb5\x1b\xd7V\xc8f-\x94\xfc\xc1]OG\x

## RSA - KEM

RSA - KEM, czyli z ang. *Key Encapsulation Mechanism* to prosty algorytm dedykowany do wymiany klucza symetrycznego. Obie strony dysponują uzgodnioną funkcją skótu H. Instancja, która chce **otrzymać** tajny klucz do komunikacji symetrycznej generuje klucze RSA i udostępnia swój klucz publiczny. Instancja, która chce wygenerować tajny klucz do komunikacji symetrycznej dysponuje kluczem publicznym instancji, która chce go otrzymać.

Instancja generująca klucz symetryczny:

1) Znajdź losową liczbę $RAND$ spełniającą warunki OAEP.

2) Oblicz: $KEY = H(RAND)$. Jeżeli trzeba, przytnij $KEY$ do odpowiedniej długości.

3) Oblicz: $CIPHERED\_KEY = RSA\_OAEP\_ENCODING(KEY, (e, n))$.

4) Wyślij $CIPHERED\_KEY$.

Instancja otrzymująca zaszyfrowany klucz symetryczny:

1) Oblicz: $KEY = RSA\_OAEP\_DECODING(CIPHERED\_KEY, (d, n))$

2) Jeżeli trzeba przytnij $KEY$ do odpowiedniej długości.

Np. AES występuje w wersji 128b, 192b i 256b. Jeżeli jako H przyjmiemy więc SHA-256, nie trzeba przycinać klucza dla algorytmu AES-256. W przeciwnym razie należy klucz odpowiednio przyciąć (z lewej lub prawej, byle obie strony tak samo) i to ta wartość staje się kluczem symetrycznym.

**Zadanie 8**

Zasymuluj takową wymianę (bez przycinania klucza).

In [268]:
r = random.randrange(0, 2**256).to_bytes(32, "big")
key = H(r)
print(key)
ck = rsa_encode(oaep_padding(key))
#print(ck)

key = oaep_remove(rsa_decode(ck), 32)
print(key)

b'5E\xa5@\r#Y\xcd\xc1\xb19f\x9cz>-\xfc`\xa92|\x82\x86\x0b.\xee\xb8\xbb\xa8F\x14U'
b'5E\xa5@\r#Y\xcd\xc1\xb19f\x9cz>-\xfc`\xa92|\x82\x86\x0b.\xee\xb8\xbb\xa8F\x14U'
